"Health Related Federal Grant Tracking"
----------------------------------------------------------------

The US government invests billions each year into the health research. The "open government" initiative provides more transparency into the governmental activities in the health related sector. Grant application data from NIH, ACF, AHRQ, CDC, HRSA, FDA, and VA are now easily accessible. With that large amount of data, we can now track the trend of the health research according to the amount of grant money. What research topics/areaes get more attractive? How have they been changing over the years? What area in the health sector will likely be the next breakthrough? I will try to explore those questions from the grant application database. 

First of all, the database called NIH ExPORTER is accesible [here](https://exporter.nih.gov/ExPORTER_Catalog.aspx). A total of 7.3 GB data were downloaded. 

In [62]:
%matplotlib inline
import matplotlib
import numpy as np
import pandas as pd
from datetime import datetime
import seaborn as sns
import os
import dill
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [22]:
from tqdm import tqdm_notebook
fns = os.listdir('./Data/PRJABS/')
fns.sort()
fns = ['./Data/PRJABS/'+x for x in fns]
proj_ABS = [pd.read_csv(fn,header = 0,dtype = {'APPLICATION_ID': str,'ABSTRACT_TEXT': str},engine = 'python') for fn in tqdm_notebook(fns)]


HBox(children=(IntProgress(value=0, max=33), HTML(value='')))

In [58]:
#add fiscal year number,fix the data type
for i in range(len(proj_ABS)):
    proj_ABS[i]['FY'] = i + 1985
    proj_ABS[i].apply(str)
#concate the whole table
proj_ABS = pd.concat(proj_ABS)

In [63]:
dill.dump(proj_ABS,open('abstracts.dill','wb'))

In [64]:
proj_ABS = dill.load(open('abstracts.dill','rb'))

,APPLICATION_ID,ABSTRACT_TEXT,FY
0,3126545,The aim of this proposal is an understanding o...,1985
1,3126549,The alternative pathway of complement activati...,1985
2,3126553,We will study virus-host cell interactions wit...,1985
3,3109190,Chronic ingestion of enthanol impairs the cont...,1985
4,3109194,The effects of ethanol exposure on central ner...,1985
